# Imports

In [7]:
import sys 

import numpy as np 
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib

import torch
import torchvision 
import torchvision.transforms as transforms

import pickle
import pandas as pd
import os

sys.path.append('../../Utils')

from SVC_Utils import *

# Load CIFAR100

In [13]:
def unpickle(file):
    with open(file, 'rb') as fo:
        res = pickle.load(fo, encoding='bytes')
    return res

transform = transforms.Compose(
    [transforms.ToTensor()])

#training data; 
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=int((trainset.__len__())/2), shuffle=True, num_workers=2)
trainloader_final=torch.utils.data.DataLoader(trainset, batch_size=trainset.__len__(), shuffle=True, num_workers=2)
#test data
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=testset.__len__(),shuffle=False, num_workers=2)

classes=None

Files already downloaded and verified
Files already downloaded and verified


In [14]:
traininputs, traintargets=load(trainloader)
testinputs, testtargets=load(testloader)
ftraininputs, ftraintargets=load(trainloader_final)

# Model Training

In [8]:
n_components=180
C_range=np.logspace(0,1,2)
gamma_range=np.logspace(-2,-1,2)

In [9]:
clfs=hp_grid(n_components=n_components, C_range=C_range, gamma_range=gamma_range)

In [6]:
#fitted_clfs=train_grid(clfs, traininputs, traintargets)

In [7]:
fitted_clfs=joblib.load('fclfs')

# Model Testing/Evaluation

In [8]:
#Stores training and testing accuracies in matrices (Rows: C_range, Cols: gamma_range)

train_accs=np.random.randn(len(C_range),len(gamma_range))
test_accs=np.random.randn(len(C_range),len(gamma_range))
test_preds=[]
k=0;

for i in range(len(C_range)):
    for j in range(len(gamma_range)):
        train_accs[i,j]=predict_eval(fitted_clfs[k], traininputs, traintargets, training=True)[1]
        preds, test_accs[i,j]=predict_eval(fitted_clfs[k], testinputs, testtargets)
        test_preds.append(preds)
        k+=1

C:  1.0 gamma:  0.01
Training Accuracy:  12.703999999999999
Testing Accuracy:  11.85
C:  1.0 gamma:  0.1
Training Accuracy:  29.415999999999997
Testing Accuracy:  19.09
C:  10.0 gamma:  0.01
Training Accuracy:  24.584
Testing Accuracy:  17.22
C:  10.0 gamma:  0.1
Training Accuracy:  58.252
Testing Accuracy:  20.14


In [12]:
idx=['C = 1','C = 10']
cols=['gamma = .01','gamma = .1']

trainacc_df=pd.DataFrame(data=train_accs, index=idx, columns=cols)
testacc_df=pd.DataFrame(data=test_accs, index=idx, columns=cols)

In [13]:
#training accuracy for C/gamma grid
trainacc_df.style.background_gradient(cmap='GnBu')

,gamma = .01,gamma = .1
C = 1,12.704,29.416
C = 10,24.584,58.252


In [14]:
#test accuracy for C/gamma grid
testacc_df.style.background_gradient(cmap='GnBu')

,gamma = .01,gamma = .1
C = 1,11.85,19.09
C = 10,17.22,20.14


# Save Model

In [ ]:
maxacc, gen=maxacc_gen(test_accs, train_accs, clfs)

In [ ]:
fn_max_acc = 'SVMCIFAR100_maxacc_proba.pkl'
fn_gen = 'SVMCIFAR100_gen_proba.pkl'

In [11]:
print(maxacc)

Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=180, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('minmaxscaler', MinMaxScaler(copy=True, feature_range=(-1, 1))), ('svc', SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])


In [15]:
save_proba(fn_max_acc, maxacc, traininputs, traintargets)
save_proba(fn_gen, gen, traininputs, traintargets)